In [2]:
import pygame
import random
import math
import sys
import mysql.connector
import json
import time
import requests

pygame.init()
pygame.font.init()

# Premenne
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
SHAPE_SIZES = [20, 30, 40]
SHAPE_COLORS = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]
SHAPE_SCORES = [12, 17, 20]
FONT_SIZE = 32
GAME_TIME = 60  # v sekundach
time_remaining = GAME_TIME
FPS = 60
MAX_SHAPES = 20
clicks = 0
click_track = 0
mouse_path = []
sum_time_between_clicks = 0
avg_time_between_clicks_in_s = 0
avg_time_between_clicks = 0
# SLEDUJEM MOUSE PATH A MOUSE MOVEMENT a VZDIALENOST MEDZI KLIKIM ale nezobrazujem to len aby som nezabudol :D 
count_clicks = 0
mouse_direction_changes = 0
previous_mouse_direction = None # zmeny po kliknuti
seconds_passed = 0
time_spent = 0
seconds = 0
speed = 0
just_clicks = 0
mouse_pos = []
time_between_c = []
# vzdialenost ktoru hrac nabehal myskou 
total_d = 0
d = 0
speeds = []
average_speed = 0 

mouse_path = []
previous_mouse_pos = (0, 0)
mouse_speed = []

# timer
#start_time = pygame.time.get_ticks()



def spawn_s(shapes):
        """Vytvori novy shape a prida ich do zoznamu"""
        x = random.randint(0, SCREEN_WIDTH)
        y = random.randint(0, SCREEN_HEIGHT)
        size = random.choice(SHAPE_SIZES)
        color = random.choice(SHAPE_COLORS)
        shapes_list = [Triangle, Circle, Star]
        shape_class = shapes_list[random.randint(0, len(shapes_list)-1)]
        shape = shape_class(x, y, size, color)
        shapes.append(shape)
        
def check_username(username):
    try:
        response = requests.get("http://cvercko.student.ki.fpvai.ukf.sk/check.php?username=" + username)
    except requests.exceptions.RequestException as e:
        print("Zadané meno už je v databaze, použite iné: ", e)
        return False
    return response.text == "0"
            
def get_username():
    username = ""
    done = False
    clock = pygame.time.Clock()
    cursor_visible = True
    cursor_time = 0
    while not done:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                if event.unicode.isalnum() or event.unicode == " ":
                    username += event.unicode
                elif event.key == pygame.K_BACKSPACE:
                    username = username[:-1]
                elif event.key in (pygame.K_RETURN, pygame.K_KP_ENTER):
                    if username == "":  # nesmie byt prazdne
                        continue
                    if check_username(username):
                        done = True
                        current_time = pygame.time.get_ticks()
                        break
                    else:
            #            username = ""
                        font = pygame.font.Font(None, FONT_SIZE)
                        text = font.render("Použivatelské meno už existuje.", True, (255, 255, 255))
                        screen.blit(text, (20, 60))
                        pygame.display.update()
                else:
                 #   username = ""
                    font = pygame.font.Font(None, FONT_SIZE)
                    text = font.render("Nesprávny znak.", True, (255, 255, 255))
                    screen.blit(text, (20, 60))
                    pygame.display.update()
        # vycistim
        screen.fill((0, 0, 0))
        
        font = pygame.font.Font(None, FONT_SIZE)
        text = font.render(f"Zadajte používateľské meno: {username}", True, (255, 255, 255))
        screen.blit(text, (20, 30))
        # cursor
        cursor_time += clock.tick(30)
        if cursor_time >= 500:
            cursor_visible = not cursor_visible
            cursor_time = 0
        if cursor_visible:
            cursor = font.render("_", True, (255, 255, 255))
            screen.blit(cursor, (20 + text.get_width(), 35))
        pygame.display.update()
    return username

def show_highscore(username, score, time_spent, clicks, just_clicks, avg_time_between_clicks_in_s ,average_speed):
    pygame.font.init()
    pygame.init()
    # meno a skore ako text
    font = pygame.font.Font(None, FONT_SIZE)
    font = pygame.font.Font(None, FONT_SIZE)
    font = pygame.font.Font(None, FONT_SIZE)
    font = pygame.font.Font(None, FONT_SIZE)
    font = pygame.font.Font(None, FONT_SIZE)
    font = pygame.font.Font(None, FONT_SIZE)
    name_text = font.render(f'Meno: {username}', True, (255, 255, 255))
    score_text = font.render(f'Skóre: {score}', True, (255, 255, 255))
    time_text = font.render(f'Čas hrania: {time_spent:.2f}', True, (255, 255, 255))
    clicks_text = font.render(f'Počet zasahov: {clicks}', True, (255, 255, 255))
    miss_text = font.render(f'Počet kliknuti: {just_clicks}', True, (255, 255, 255))
    avg_text = font.render(f'Čas medzi kliknutiami: {avg_time_between_clicks_in_s:.2f}', True, (255, 255, 255))  
    # vycisti obrazovku
    screen.fill((0,0,0))

    # zobrazi meno a skore
    screen.blit(name_text, (10, 10))
    screen.blit(score_text, (10, 50))
    screen.blit(time_text, (10, 90))
    screen.blit(clicks_text, (10, 130))
    screen.blit(miss_text, (10, 170))
    screen.blit(avg_text, (10, 210))
    pygame.display.update()
    
    time.sleep(7) #po dohrati hry počka 7 sekund, ukaže skore a nasledne zavrie a posle do db
    # je to vyhodnejsie riesenie...
    
    send_results_to_db(username, score, time_spent, clicks, just_clicks, avg_time_between_clicks_in_s, average_speed)
    print('dáta úspešne odoslané')
   
def game_over(username, score, surface):
    font = pygame.font.Font(None, FONT_SIZE)
    text = font.render(f'{username}, konečné skóre je: {score} za {time_spent:.2f} sekúnd s {clicks} zasahmi a netrafil si sa {missed_clicks} krat, avg TBC {average_time_between_clicks_in_seconds:.2f} s  ', True, (255, 255, 255))
    text_rect = text.get_rect()
    text_rect.center = (text,SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2)
    surface.blit(text, text_rect)
    show_highscore(username, score, time_spent, clicks, just_clicks,avg_time_between_clicks_in_s,average_speed)
    
def send_results_to_db(username, score, time_spent, clicks, just_clicks, avg_time_between_clicks_in_s, average_speed):
    if len(mouse_speed) > 0:
        average_speed = sum(mouse_speed) / len(mouse_speed)
        x = requests.get('http://cvercko.student.ki.fpvai.ukf.sk/test.php?username='+username+
                                                                     '&score='+str(score)+
                                                                     '&time_spent='+str(time_spent)+
                                                                     '&clicks='+str(clicks)+
                                                                     '&just_clicks='+str(just_clicks)+
                                                                     '&avg_time_between_clicks_in_s='+str(avg_time_between_clicks_in_s)+
                                                                     '&average_speed='+str(average_speed)
                                                                     ,verify = False)
    print(x.text)
class Shape:
    """Zakladne shapy"""
    def __init__(self, x, y, size, color):
        self.x = x
        self.y = y
        self.size = size
        self.color = color
        self.speed = random.uniform(1, 6)
        self.direction = random.uniform(0, 360)   
        
    def collidepoint(self, point):
        """Kolizie vramci hernej plochy"""
        x, y = point
        if x < self.x - self.size or x > self.x + self.size:
            return False
        if y < self.y - self.size or y > self.y + self.size:
            return False
        return True    
    
    def remove(self, shapes):
        """odstrani shape na ktory hrac klikol"""
        shapes.remove(self)
     
    def move(self):
        """nahodny smer pohybu"""
        self.x += self.speed * math.cos(math.radians(self.direction))
        self.y += self.speed * math.sin(math.radians(self.direction))

        # davam pozor aby neusiel shape
        if self.x < 0 or self.x > SCREEN_WIDTH:
            self.direction = (180 - self.direction) % 360
        if self.y < 0 or self.y > SCREEN_HEIGHT:
            self.direction = (- self.direction) % 360

    def draw(self, surface):
        """nakresli shapy podla danych parametrov"""
        pass  # podtriedy maju udane presnejsie parametre            
    
class Triangle(Shape):
    def __init__(self, x, y, size, color):
        super().__init__(x, y, size, color)
        
    def draw(self, surface):
        """nakresli Trojuholnik"""
        points = [(self.x, self.y - self.size),
                  (self.x - self.size, self.y + self.size),
                  (self.x + self.size, self.y + self.size)]
        pygame.draw.polygon(surface, self.color, points)

class Circle(Shape):
    def __init__(self, x, y, size, color):
        super().__init__(x, y, size, color)
        
    def draw(self, surface):
        """Nakreslim Kruh"""
        pygame.draw.circle(surface, self.color, (self.x, self.y), self.size)
    
class Star(Shape):
    def __init__(self, x, y, size, color):
        super().__init__(x, y, size, color)
    
    def draw(self, surface):
        """Nakreslim akoze hviezdu"""
        points = [(self.x, self.y - self.size),
          (self.x - self.size/2, self.y - self.size/2),
          (self.x - self.size, self.y),
          (self.x - self.size/2, self.y + self.size/2),
          (self.x, self.y + self.size),   
          (self.x + self.size/2, self.y + self.size/2),
          (self.x + self.size, self.y),
          (self.x + self.size/2, self.y - self.size/2)
         ]
        pygame.draw.polygon(surface, self.color, points)

def main():
    #pygame.init()
    print('fungujem') #treba to tu nechat...
# pred zacatim hry potrebujem pouzivatelske meno
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
username = get_username() #zadam username az potom spustim timery
start_time = pygame.time.get_ticks()
previous_click_time = pygame.time.get_ticks() #pomocne na vypocty
previous_mouse_position = pygame.mouse.get_pos()
clock = pygame.time.Clock()
pygame.display.set_caption('ZP Clicker LC')
# vytvorim shapy a dam ich do zoznamu nahodne
shapes = []
for _ in range(3):
    x = random.randint(0, SCREEN_WIDTH)
    y = random.randint(0, SCREEN_HEIGHT)
    size = random.choice(SHAPE_SIZES)
    color = random.choice(SHAPE_COLORS)
    if random.random() < 0.25:
        shape = Triangle(x, y, size, color)
    elif random.random() < 0.50:
        shape = Star(x, y, size, color)
    else:
        shape = Circle(x, y, size, color)
    shapes.append(shape)

# vytvorim font na zobrazenie udajov
font = pygame.font.Font(None, FONT_SIZE)

font = pygame.font.Font(None, FONT_SIZE)
# game loop 
last_click_time = pygame.time.get_ticks()
score = 0
#start_time = pygame.time.get_ticks()
running = True
clock.tick(FPS)  
#********* tu zacina hra akoby
while running: # and (pygame.time.get_ticks() - start_time) < GAME_TIME*1000:
    
    current_time = pygame.time.get_ticks()
    prev_time = current_time
    current_pos = pygame.mouse.get_pos()
    prev_pos = current_pos
    
    seconds += clock.get_time() / 1000
    if seconds >= 10:
        # kazdych 10 sekund ubudnu 2 shapy a resetujem tych 10 sekund aby sa to opakovalo
        seconds = 0
        MAX_SHAPES -= 2
     # timer
    mouse_position = pygame.mouse.get_pos()
    seconds_passed = clock.get_time() / 1000
    time_spent += seconds_passed
    # Sledujem Kliknutia, movement etc.
    for event in pygame.event.get():
        if event.type == pygame.QUIT:   
            running = False
            show_highscore(username, score, time_spent, clicks, just_clicks, avg_time_between_clicks_in_s,average_speed)   
        elif event.type == pygame.MOUSEBUTTONDOWN:
            # kliknutia myskou
            current_position = pygame.mouse.get_pos()
            current_mouse_position = pygame.mouse.get_pos()
             # smer pohybu mysky
            x_diff = current_mouse_position[0] - previous_mouse_position[0]
            y_diff = current_mouse_position[1] - previous_mouse_position[1]
            current_mouse_direction = (x_diff, y_diff)
            # porovnam predosle s aktualnym
            if previous_mouse_direction and current_mouse_direction != previous_mouse_direction:
                mouse_direction_changes += 1
             # updatnem pozicie
            previous_mouse_position = current_mouse_position
            previous_mouse_direction = current_mouse_direction
                # clicktrack updatnem
            click_track += 1
            # print(click_track)
            distance = math.sqrt((current_mouse_position[0] - previous_mouse_position[0]) ** 2 + (current_mouse_position[1] - previous_mouse_position[1]) ** 2)
            previous_mouse_position = current_mouse_position
            mouse_x, mouse_y = pygame.mouse.get_pos()
      #      current_time = pygame.time.get_ticks()
            just_clicks += 1 #kliknutia
            time_between_clicks = current_time - previous_click_time # tu počítam AVG medzi klikmi
            sum_time_between_clicks += time_between_clicks
            previous_click_time = current_time
            if just_clicks > 0: #kliknuti musi byt viac ako 1 
                avg_time_between_clicks = sum_time_between_clicks / just_clicks
                avg_time_between_clicks_in_s = avg_time_between_clicks / 1000
            else:
                avg_time_between_clicks = 0
                avg_time_between_clicks_in_s = 0
                
            for shape in shapes:
                if shape.collidepoint(event.pos):
                    shape.remove(shapes)
                    score += SHAPE_SCORES[SHAPE_SIZES.index(shape.size)]
                    clicks += 1       
            else:
                score -= 5 #odrata 5 score ked hrac missne 
                
        elif event.type == pygame.MOUSEMOTION:
            current_mouse_position = event.pos
            mouse_x, mouse_y = pygame.mouse.get_pos()
            if mouse_x >= 0 and mouse_x <= SCREEN_WIDTH and mouse_y >= 0 and mouse_y <= SCREEN_HEIGHT:
                x1, y1 = previous_mouse_position
                x2, y2 = current_mouse_position
                if x1 != x2 or y1 != y2:
                    previous_mouse_position = current_mouse_position
                    mouse_path.append(current_mouse_position)
                
    for i in range(1, len(mouse_path)):
        x_distance = mouse_path[i][0] - mouse_path[i-1][0]
        y_distance = mouse_path[i][1] - mouse_path[i-1][1]
        d = math.sqrt(x_distance**2 + y_distance**2)
        total_d += d
        mouse_speed.append(d)

    if time_remaining <= 0 or score >= 1100: #ak dosiahne 1300 je dobry
        running = False
        show_highscore(username, score, time_spent, clicks, just_clicks, avg_time_between_clicks_in_s,average_speed)
        print('čas vypršal')
        print(time_spent)
        
        # Update casu  
 #   current_time = pygame.time.get_ticks()
    time_elapsed = (current_time - start_time) / 1000
    time_remaining = GAME_TIME - time_elapsed
    # update displeja
    pygame.display.update()                 
# spawnuje novy shape kazdu chvilku
    elapsed_time = pygame.time.get_ticks() - start_time
    if elapsed_time % 200 < 16 and len(shapes) < MAX_SHAPES:  #  16 ms pri 60 fps
        spawn_s(shapes)
        
    # pozicie objektov
    for shape in shapes:
        shape.move()

    # nakreslim na obrazovku
    screen.fill((255, 255, 255))
    for shape in shapes:
        shape.draw(screen)
        
   # zobrazim skore atd
    num_shapes = len(shapes)
    text = font.render("Počet objektov: " + str(num_shapes), True, (0, 0, 0))
    text_rect = text.get_rect()
    text_rect.center = (680,20)
    screen.blit(text, text_rect)
    
    text = font.render(f'Skóre: {score}', True, (0, 0, 0))
     # Renderujem time ako string
    time_string = "Čas: {:.1f}".format(time_remaining)
    time_text = font.render(time_string, True, (0, 0, 0))
    text_rect = text.get_rect()
    screen.blit(text, text_rect)
    screen.blit(time_text, (380,8))
    # Update dispeja
    pygame.display.update()
    clock.tick(60)

if __name__ == '__main__':
    main()
    pygame.quit()

Connected successfullyArray
(
    [username] => dddddddddddddd
    [score] => 279
    [time_spent] => 17.5529999999998
    [clicks] => 28
    [just_clicks] => 44
    [avg_time_between_clicks_in_s] => 0.38622727272727275
    [average_speed] => 5.943215511938775
)

dáta úspešne odoslané
fungujem
